In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'omkar_proj.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from account.models import (
    User,
    Farmer,
    Rental,
    Investor,
    UserAndInvestor
)
from warehouse.models import (
    Gala,
    Property,
    Company,
)
from django.db.models import (
    Count,
    F,
    Sum,
    OuterRef,
    Prefetch,
    
)

from contract.models import (
    Contract,
    Investor as ContractInvestor,
    Rental as ContractRental,
    Farmer as ContractFarmer
)
from account.models import User

In [41]:

# # rf = Gala.objects.filter(is_allotted=True).values('uid','gala_number'
# #                                                   ).annotate(
# #     total_not_allotted_galas = Count('warehouse__uid'))
# # rf
# # rf = Property.objects.filter(company__name__exact='OMKAR').values('uid','property_name'
# #                                     ).annotate(get_galas = Gala.objects.filter(is_allotted=False))

# rf = Property.objects.filter(company__name__exact='OMKAR').values('uid','property_name'
#                                                                  ).annotate(get_galas= Property.objects.prefetch_related('get_gala').filter(get_gala__is_allotted=False))



In [42]:
rf = Company.objects.get(name='Omkar').get_properties.all().values('uid','property_name'
            ).annotate(total_remaining_galas =Gala.objects.filter(warehouse__uid=OuterRef('uid'),is_allotted=False
                                                                 ).values('warehouse__uid'
                    ).annotate(total_remaining_galas=Count('id')).values('total_remaining_galas'))
rf
                               


<QuerySet [{'uid': UUID('5ec6fd54-9257-487c-8e01-8d171b3c74a5'), 'property_name': 'Omkar Warehouse 2', 'total_remaining_galas': 3}, {'uid': UUID('9bbc8a5f-e1a3-434f-b1a7-6e7b820492c2'), 'property_name': 'Omkar Warehouse 1', 'total_remaining_galas': 1}]>

In [43]:
# user_type = "Investor"
# if user_type == "Investor":
get_investor_gala_contract_detail = Company.objects.get(name="Omkar"

                ).get_properties.get(uid="9bbc8a5f-e1a3-434f-b1a7-6e7b820492c2").get_gala.filter(

                        gala_investor_contract_detail__owner__is_superuser=True,

                ).values("gala_investor_contract_detail__owner","gala_investor_contract_detail__user")

get_rental_contract_detail_w_owner = Company.objects.get(name="Omkar"
                    ).get_properties.get(uid = "9bbc8a5f-e1a3-434f-b1a7-6e7b820492c2").get_gala.filter(

                        gala_rental_contract_detail__owner__is_superuser = True,
                        gala_rental_contract_detail__user__groups__name = "Rental"
                        ).values("gala_rental_contract_detail__owner","gala_rental_contract_detail__user")
    



In [44]:
df = read_frame(get_investor_gala_contract_detail)


In [45]:
df


,gala_investor_contract_detail__owner,gala_investor_contract_detail__user
0,1,13
1,1,44


In [46]:
gf = read_frame(get_rental_contract_detail_w_owner)

In [47]:
gf

,gala_rental_contract_detail__owner,gala_rental_contract_detail__user
0,1,2
1,1,11


In [48]:
get_company_name = Company.objects.get(name="Omkar"
                        ).get_properties.get(uid="9bbc8a5f-e1a3-434f-b1a7-6e7b820492c2"
                            ).get_gala.filter(is_allotted=False)#.select_related("warehouse","uid").values("warehouse__uid")
#     "warehouse__property_name","warehouse__property_survey_number","gala_number")

tf = read_frame(get_company_name)


In [49]:
tf

,id,uid,created_at,updated_at,warehouse,gala_number,is_allotted
0,6,a5c94524-be8a-46ba-a5ca-5613cf90eaba,2022-12-20 08:52:59.560840+00:00,2022-12-20 08:52:59.560922+00:00,Omkar Warehouse 1,dff7567c-bdee-4b21-9f55-59644fce0cbd,False


In [50]:
get_company_name = Company.objects.get(name="Omkar").get_properties.filter(get_gala__is_allotted=False).count()


get_company_name


4

In [51]:
# total_remaining_gala_count

In [52]:
# data

In [3]:
from account.models import (
    User,
    Farmer,
    Rental,
    Investor,
    UserAndInvestor
)
from warehouse.models import (
    Gala,
    Property,
    Company,
)
from django.db.models import (
    Count,
    F,
    Sum,
    OuterRef,
    Prefetch,
    
)

from contract.models import (
    Contract,
    Investor as ContractInvestor,
    Rental as ContractRental,
    
    Farmer as ContractFarmer
)
from account.models import User

In [26]:
Investor.objects.filter(belong_to__name = "Omkar").values(
                "id","user_uid","username","first_name","last_name","address","phone","address",
                "city","zip_code","birth_date"
                ).annotate(
                    total_galas = ContractInvestor.objects.filter(
                        user__user_uid = OuterRef("user_uid")).values("user__user_uid"
                ).annotate(total_galas=Count("id",distinct=True)).values("total_galas"),
                    total_rentals = ContractRental.objects.filter(
                        owner__user_uid = OuterRef("user_uid")).values("user__user_uid"
                ).annotate(total_galas=Count("id",distinct=True)).values("total_galas")
            )

<QuerySet [{'id': 8, 'user_uid': UUID('0591cd66-fd57-4b10-b372-5c21735bdb90'), 'username': 'mayank', 'first_name': 'Mayank', 'last_name': 'Motiyani', 'address': 'Rajendra Nagar', 'phone': '8560025508', 'city': 'Indore', 'zip_code': '312001', 'birth_date': datetime.date(1995, 12, 14), 'total_galas': 2, 'total_rentals': 1}]>

In [23]:
ContractInvestor.objects.filter(user_id = 8).values("id").annotate()

<PolymorphicQuerySet [{'id': 5}, {'id': 3}]>

In [12]:
Gala.objects.filter(warehouse__company__name="Omkar").select_related("gala_investor_contract_detail","gala_rental_contract_detail")

<QuerySet [<Gala: ced84a5a-d56c-45b6-8787-d649a5c93252>, <Gala: 94c33451-5cae-45cb-8e26-ccdb63bca7da>, <Gala: ac9e4b8f-4a62-4e9a-900f-baf2035c0f6c>, <Gala: 10fbc2c7-8635-4631-9648-3b6a51dff889>, <Gala: 68d07089-5f17-4b35-b079-0f0830f2fca2>, <Gala: e9b90a6b-4ea9-4fca-95fd-22daae3fcadd>]>

In [15]:
UserAndInvestor.objects.filter()

<QuerySet [<UserAndInvestor: admin>, <UserAndInvestor: snippy-aqua-ladybird>, <UserAndInvestor: shaggy-lemon-deer>, <UserAndInvestor: skinny-xanthic-tamarin>, <UserAndInvestor: stealthy-platinum-lion>, <UserAndInvestor: whiny-white-ostrich>, <UserAndInvestor: shaggy-maroon-audemer>, <UserAndInvestor: baggy-olive-dragonfly>, <UserAndInvestor: crabby-asparagus-tzu>, <UserAndInvestor: tacky-seashell-hummingbird>, <UserAndInvestor: gloppy-tomato-birman>, <UserAndInvestor: cranky-khaki-budgerigar>, <UserAndInvestor: snappy-burgundy-indri>, <UserAndInvestor: stuffy-cardinal-binturong>, <UserAndInvestor: foggy-purple-koala>, <UserAndInvestor: lanky-lavender-leopard>, <UserAndInvestor: fuzzy-chartreuse-boar>, <UserAndInvestor: silly-apricot-dingo>, <UserAndInvestor: snoopy-ivory-frigatebird>, <UserAndInvestor: woolly-peach-kangaroo>, '...(remaining elements truncated)...']>

In [19]:
ContractRental.objects.all()

<PolymorphicQuerySet [<Rental: Leave_and_License>, <Rental: Leave_and_License>, <Rental: Leave_and_License>]>

In [45]:
Gala.objects.filter(
    warehouse__company__name="Omkar",
    is_allotted_to_rental=True).values(
    "gala_rental_contract_detail__owner__user_uid"
).annotate(
    number_of_rentals = ContractRental.objects.filter(owner__user_uid=OuterRef("gala_rental_contract_detail__owner__user_uid")).values("owner").annotate(number_of_rentals = Count("owner")).values('number_of_rentals')
)

<QuerySet [{'gala_rental_contract_detail__owner__user_uid': UUID('12b0fa6f-3739-4489-b6e4-67ac12b677bf'), 'number_of_rentals': 1}, {'gala_rental_contract_detail__owner__user_uid': UUID('2d3a2eff-636c-4491-8908-1f424657c085'), 'number_of_rentals': 1}, {'gala_rental_contract_detail__owner__user_uid': UUID('f4056ac8-635c-4907-aa45-482fc51174c3'), 'number_of_rentals': 1}]>

In [36]:
ContractRental.objects.filter(owner_id=1).values("owner")

<PolymorphicQuerySet [{'owner': 1}]>

In [58]:
Gala.objects.filter(warehouse__company__name="Omkar",is_allotted_to_rental=True).select_related("gala_rental_contract_detail__owner","gala_rental.annotate(
    number_of_rentals = ContractRental.objects.filter(owner__user_uid=OuterRef("gala_rental_contract_detail__owner__user_uid")).values("owner").annotate(number_of_rentals = Count("owner")).values('number_of_rentals')
)

<QuerySet [{'gala_rental_contract_detail': 56, 'number_of_rentals': 1}, {'gala_rental_contract_detail': 57, 'number_of_rentals': 1}, {'gala_rental_contract_detail': 54, 'number_of_rentals': 1}]>

In [51]:
Gala.objects.filter(
    warehouse__company__name="Omkar",
    is_allotted_to_rental=True).values(
    "gala_rental_contract_detail__owner__user_uid",
    "gala_rental_contract_detail__owner__username",
).annotate(
    number_of_rentals = ContractRental.objects.filter(owner__user_uid=OuterRef("gala_rental_contract_detail__owner__user_uid")).values("owner").annotate(number_of_rentals = Count("owner")).values('number_of_rentals')
)

<QuerySet [{'gala_rental_contract_detail__owner__user_uid': UUID('12b0fa6f-3739-4489-b6e4-67ac12b677bf'), 'gala_rental_contract_detail__owner__username': 'skinny-xanthic-tamarin', 'number_of_rentals': 1}, {'gala_rental_contract_detail__owner__user_uid': UUID('2d3a2eff-636c-4491-8908-1f424657c085'), 'gala_rental_contract_detail__owner__username': 'admin', 'number_of_rentals': 1}, {'gala_rental_contract_detail__owner__user_uid': UUID('f4056ac8-635c-4907-aa45-482fc51174c3'), 'gala_rental_contract_detail__owner__username': 'shaggy-lemon-deer', 'number_of_rentals': 1}]>

In [59]:
Gala.objects.filter(warehouse__company__name="Omkar",is_allotted_to_rental=True).select_related("gala_rental_contract_detail")

<QuerySet [<Gala: ced84a5a-d56c-45b6-8787-d649a5c93252>, <Gala: ac9e4b8f-4a62-4e9a-900f-baf2035c0f6c>, <Gala: 10fbc2c7-8635-4631-9648-3b6a51dff889>]>

In [8]:
queryset = (
        Gala.objects.filter(warehouse__company__name="Omkar",is_allotted_to_rental=True)
        .prefetch_related(
            Prefetch(
                'gala_rental_contract_detail', to_attr='number_of_rentals',
                queryset=ContractRental.objects.all().annotate(
                    number_of_rentals = Count('id', distinct=True),
                )
            ),
            
        )
    )


In [9]:
queryset

<QuerySet [<Gala: ced84a5a-d56c-45b6-8787-d649a5c93252>, <Gala: ac9e4b8f-4a62-4e9a-900f-baf2035c0f6c>, <Gala: 10fbc2c7-8635-4631-9648-3b6a51dff889>]>

In [10]:
# Prefetch(
#                 'followers', to_attr='prefetched_followers',
#                 queryset=User.objects.all()
#             ),

In [11]:
from django_pandas.io import read_frame


ModuleNotFoundError: No module named '_bz2'

In [13]:
!pip install bz2

ERROR: Could not find a version that satisfies the requirement bz2 (from versions: none)
ERROR: No matching distribution found for bz2
You should consider upgrading via the '/home/python-25/Desktop/Omkar-Development/venv/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
df = read_frame(queryset)

In [ ]:
df

In [3]:
from account.models import (
    User,
    Farmer,
    Rental,
    Investor,
    UserAndInvestor
)
from warehouse.models import (
    Gala,
    Property,
    Company,
)
from django.db.models import (
    Count,
    F,
    Sum,
    OuterRef,
    Prefetch,
    
)

from contract.models import (
    Contract,
    Investor as ContractInvestor,
    Rental as ContractRental,
    Farmer as ContractFarmer
)
from account.models import User

In [4]:
get_rental = ContractRental.objects.filter(user_id=5).values("gala__warehouse__property_name","gala__warehouse__company__name","gala__gala_number","user__username","gala__warehouse__property_type")

In [5]:
get_rental

<PolymorphicQuerySet [{'gala__warehouse__property_name': 'Omkar Warehouse 1', 'gala__warehouse__company__name': 'Omkar', 'gala__gala_number': 'e9b90a6b-4ea9-4fca-95fd-22daae3fcadd', 'user__username': 'snappy-cardinal-argentino', 'gala__warehouse__property_type': 'COLD STORAGE'}, {'gala__warehouse__property_name': 'Omkar Warehouse 2', 'gala__warehouse__company__name': 'Omkar', 'gala__gala_number': '94c33451-5cae-45cb-8e26-ccdb63bca7da', 'user__username': 'snappy-cardinal-argentino', 'gala__warehouse__property_type': 'RCC'}, {'gala__warehouse__property_name': 'Omkar Warehouse 2', 'gala__warehouse__company__name': 'Omkar', 'gala__gala_number': 'ac9e4b8f-4a62-4e9a-900f-baf2035c0f6c', 'user__username': 'snappy-cardinal-argentino', 'gala__warehouse__property_type': 'RCC'}]>

In [6]:
get_rental_qs = Rental.objects.filter(belong_to__name="Omkar").values(
                "id","user_uid","username"
            ).annotate(
                total_number_of_galas = ContractRental.objects.filter(
                    user__user_uid = OuterRef("user_uid")
                ).values("user__user_uid").annotate(
                    total_number_of_galas = Count("user__user_uid")
                ).values("total_number_of_galas"),
            )

In [7]:
get_rental_qs

<QuerySet [{'id': 2, 'user_uid': UUID('dfef7ec8-d0fe-4e9f-abb1-16187c184320'), 'username': 'Mayank', 'total_number_of_galas': 1}, {'id': 5, 'user_uid': UUID('51f9c7eb-8f1b-4fcc-9124-58831235d387'), 'username': 'snappy-cardinal-argentino', 'total_number_of_galas': 3}, {'id': 14, 'user_uid': UUID('1df8502b-66a6-4486-ae21-7e19c3e7bd0a'), 'username': 'randy-crimson-caracal', 'total_number_of_galas': 1}, {'id': 15, 'user_uid': UUID('4acd799f-7f94-42de-9b6d-1e80b5c5e6b3'), 'username': 'cloudy-champagne-zebu', 'total_number_of_galas': None}, {'id': 19, 'user_uid': UUID('21cc2354-694b-44fd-b9cb-659f1117ac55'), 'username': 'slimy-platinum-tapir', 'total_number_of_galas': None}, {'id': 21, 'user_uid': UUID('0fccdc06-c2e6-45b2-9b91-0100b976e6d3'), 'username': 'squirrely-razzmatazz-moose', 'total_number_of_galas': None}, {'id': 28, 'user_uid': UUID('e7b74f37-29df-4650-b622-7b56626bcdd8'), 'username': 'tasty-platinum-hyrax', 'total_number_of_galas': None}, {'id': 37, 'user_uid': UUID('4e16d99b-f073

In [26]:
from django.db.models import (
    ExpressionWrapper,
    IntegerField,
    Window
)

In [27]:
ContractRental.objects.filter(user__user_uid = '51f9c7eb-8f1b-4fcc-9124-58831235d387').values("gala__warehouse")

<PolymorphicQuerySet [{'gala__warehouse': 8}, {'gala__warehouse': 9}, {'gala__warehouse': 9}]>

In [36]:
ContractRental.objects.filter(
    user__user_uid = '51f9c7eb-8f1b-4fcc-9124-58831235d387'
).values(
    "gala__warehouse"
).annotate(Count("gala__warehouse",distinct=True))

<PolymorphicQuerySet [{'gala__warehouse': 8, 'gala__warehouse__count': 1}, {'gala__warehouse': 9, 'gala__warehouse__count': 1}]>

In [57]:
from django.db import models

In [58]:
ContractRental.objects.filter(
    user__user_uid = '51f9c7eb-8f1b-4fcc-9124-58831235d387'
).values(
    "gala__warehouse"
).annotate(Count("gala__warehouse",distinct=True)).annotate(total_warehouse = ExpressionWrapper(
        Count('gala__warehouse_id',distinct=True), output_field=IntegerField())).values("total_warehouse").annotate(points=models.Sum('total_warehouse'))

FieldError: Cannot compute Sum('total_warehouse'): 'total_warehouse' is an aggregate

In [ ]:
# Window(expression=Sum(Count('comments__id')), partition_by=[F('group')])


In [ ]:
# Note:  CustomUser is what get_user_model() returns
CustomUser.objects.annotate(post_count=Count('posts'), comment_count=Count('comment'),
        total=Sum(F('post_count') * 10 + F('comment_count'))
    ).order_by('-post_count', '-comment_count', 'username')

In [ ]:
CustomUser.objects.annotate(
    post_count=Count('posts'), 
    comment_count=Count('comment'),
    total=F('post_count') * 10 + F('comment_count')
).order_by('-post_count', '-comment_count', 'username')

In [54]:
ContractRental.objects.filter(
    user__user_uid = '51f9c7eb-8f1b-4fcc-9124-58831235d387'
).values(
    "gala__warehouse"
).annotate(Count("gala__warehouse",distinct=True))

<PolymorphicQuerySet [{'gala__warehouse': 8, 'gala__warehouse__count': 1}, {'gala__warehouse': 9, 'gala__warehouse__count': 1}]>

In [133]:
get_rental_qs = Rental.objects.filter(belong_to__name="Omkar").values(
                "id","user_uid","username"
            ).annotate(
                total_number_of_galas = ContractRental.objects.filter(
                    user__user_uid = OuterRef("user_uid")
                ).values('gala__warehouse_id').annotate(gala_warehouse = Count("gala__warehouse_id",distinct=True)).values("gala_warehouse").annotate(
                    total_number_of_galas=ExpressionWrapper(
                        Count("gala__warehouse_id"),
                        output_field=models.IntegerField()
                    )
                ).values('total_number_of_galas')).annotate(
                total_number_of_warehouse = ContractRental.objects.filter(
                    user__user_uid = OuterRef("user_uid")
                ).values('gala__warehouse_id').annotate(gala_warehouse = Count("gala__warehouse_id",distinct=True)).values("gala_warehouse").annotate(
                    number_of_warehouse=ExpressionWrapper(
                        Count("gala__warehouse_id",distinct=True),
                        output_field=models.IntegerField()
                    )
                ).values('number_of_warehouse') 
                                )
                            

In [134]:
get_rental_qs

<QuerySet [{'id': 2, 'user_uid': UUID('dfef7ec8-d0fe-4e9f-abb1-16187c184320'), 'username': 'Mayank', 'total_number_of_galas': 1, 'total_number_of_warehouse': 1}, {'id': 5, 'user_uid': UUID('51f9c7eb-8f1b-4fcc-9124-58831235d387'), 'username': 'snappy-cardinal-argentino', 'total_number_of_galas': 3, 'total_number_of_warehouse': 2}, {'id': 14, 'user_uid': UUID('1df8502b-66a6-4486-ae21-7e19c3e7bd0a'), 'username': 'randy-crimson-caracal', 'total_number_of_galas': 1, 'total_number_of_warehouse': 1}, {'id': 15, 'user_uid': UUID('4acd799f-7f94-42de-9b6d-1e80b5c5e6b3'), 'username': 'cloudy-champagne-zebu', 'total_number_of_galas': 0, 'total_number_of_warehouse': 0}, {'id': 19, 'user_uid': UUID('21cc2354-694b-44fd-b9cb-659f1117ac55'), 'username': 'slimy-platinum-tapir', 'total_number_of_galas': 0, 'total_number_of_warehouse': 0}, {'id': 21, 'user_uid': UUID('0fccdc06-c2e6-45b2-9b91-0100b976e6d3'), 'username': 'squirrely-razzmatazz-moose', 'total_number_of_galas': 0, 'total_number_of_warehouse': 

In [ ]:
.annotate(total_number_of_warehouse = ContractRental.objects.filter(user__user_uid = OuterRef("user_uid")).values(
"gala__warehouse_id").annotate(total_number_of_warehouse = Count(F("gala__warehouse_id"))).values("total_number_of_warehouse"))

In [ ]:
"51f9c7eb-8f1b-4fcc-9124-58831235d387"

In [84]:
ContractRental.objects.filter(user__user_uid = '51f9c7eb-8f1b-4fcc-9124-58831235d387').values(
"gala__warehouse_id").annotate(total_number_of_warehouse = Count(F("gala__warehouse_id"),distinct=True)).values("total_number_of_warehouse")

<PolymorphicQuerySet [{'total_number_of_warehouse': 1}, {'total_number_of_warehouse': 1}]>

In [ ]:
responses = AnswerResponse.objects
        .filter(survey__pk=models.OuterRef('pk'))
        .annotate(
            points=models.ExpressionWrapper(
                models.F('realcoef') * models.F('answer__question__points'),
                output_field=models.IntegerField()
            )
        )
        .values('survey__pk') 
        .annotate(
            total=models.Sum(models.F('points'))
        )
        .values('total')

<PolymorphicQuerySet [{'points': 2}]>

In [ ]:
.annotate(
    points=ExpressionWrapper(
        Count("gala__warehouse_id"),
        output_field=models.IntegerField()
    )
).values('points') 

In [129]:
ContractRental.objects.filter(
                    user__user_uid = "51f9c7eb-8f1b-4fcc-9124-58831235d387"
                ).values('gala__warehouse_id').annotate(gala_warehouse = Count("gala__warehouse_id",distinct=True)).values("gala_warehouse").annotate(
                    total_number_of_galas=ExpressionWrapper(
                        Count("gala__warehouse_id"),
                        output_field=models.IntegerField()
                    )
                ).values('total_number_of_galas') 

<PolymorphicQuerySet [{'total_number_of_galas': 0}]>